In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, CategoryParameter, expand_plist, FieldSelector, Parameter, wwr
from besos.problem import EPProblem
plt.rcParams['figure.figsize'] = [8, 8]

In [2]:
filename = 'model_final_triple'
building = ef.get_building('/home/ict4bd/'+filename+'.idf')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ict4bd/model_final_triple.idf'

In [ ]:
[materials.Name for materials in building.idfobjects["MATERIAL"]] # get a list of the Name property of all IDF objects of class MATERIAL

In [ ]:
building.idfobjects["BUILDING"][0]

In [ ]:
building.idfobjects["MATERIAL"]

In [ ]:
insulation = FieldSelector(class_name='Material',\
                           object_name='XPS Extruded Polystyrene  - CO2 Blowing_.2',\
                           field_name='Thickness')
ach_bedroom = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Bedroom Nat Vent', field_name='Air Changes per Hour')
ach_bathroom=FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Bathroom Nat Vent', field_name='Air Changes per Hour')
ach_kitchen=FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Kitchen Nat Vent', field_name='Air Changes per Hour')
shading_bedroom = FieldSelector(class_name='WindowShadingControl',object_name='1001-Block1:Bedroom', field_name='Setpoint')
shading_bathroom = FieldSelector(class_name='WindowShadingControl',object_name='1002-Block1:Bathroom', field_name='Setpoint')
shading_kitchen = FieldSelector(class_name='WindowShadingControl',object_name='1003-Block1:Kitchen', field_name='Setpoint')

In [ ]:
zero_to_one_exclusive = CategoryParameter(options=[0.00001, 0.10, 0.20])
ach_parameters = CategoryParameter(options=[0,5])
shading_parameters = CategoryParameter(options=[120,2000])

In [ ]:
insulation_param = Parameter(selector=insulation,
                                 value_descriptor=zero_to_one_exclusive,
                                 name='Insulation Thickness')

ach_bedroom_param = Parameter(selector=ach_bedroom,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Bedroom')

ach_bathroom_param = Parameter(selector=ach_bathroom,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Bathroom')

ach_kitchen_param = Parameter(selector=ach_kitchen,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Kitchen')

shading_bedroom_param=Parameter(selector=shading_bedroom,
                                 value_descriptor=shading_parameters,
                                 name='Shading Bedroom')

shading_bathroom_param=Parameter(selector=shading_bathroom,
                                 value_descriptor=shading_parameters,
                                 name='Shading Bathroom')

shading_kitchen_param=Parameter(selector=shading_kitchen,
                                 value_descriptor=shading_parameters,
                                 name='Shading Kitchen')

In [ ]:
window_to_wall = wwr(RangeParameter(0.1,0.9)) # use a special shortcut to get the window-to-wall parameter
window_to_wall

In [ ]:
#previous = building.idfobjects["BUILDING"][0].obj[2]
#building.idfobjects["BUILDING"][0].obj[2] = [0.0, 90.0, 180.0, 270.0]

orientation_param = FieldSelector(class_name='Building',\
                           object_name='Building',\
                           field_name='North Axis')
orientation_parameters = CategoryParameter(options=[0.0, 90.0, 180.0, 270.0])
orientation = Parameter(selector = orientation_param, value_descriptor = orientation_parameters, name = 'Orientation')

orientation

In [ ]:
# Add all the parameters to a single parameters object.
#parameters = [insulation_param] + [window_to_wall]
parameters = [window_to_wall,insulation_param,orientation,ach_bedroom_param,ach_bathroom_param,ach_kitchen_param,shading_bedroom_param,shading_bathroom_param,shading_kitchen_param] 

# Let us try to optimize the Cooling and the Heating of facility at the same time
objectives = ['Electricity:Facility','DistrictCooling:Facility'] 
#'InteriorLights:Electricity' has correlation circa 1 with ELectricity:Facility, we remove it from the obj
# These are hourly values, default is sum all the hourly values

# Construct the problem 
problem = EPProblem(parameters, objectives) # Make a problem instance from the parameters and objectives

In [ ]:
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=12)

#samples = pd.read_csv(filename+'.csv').drop(['Electricity:Facility','DistrictCooling:Facility'], axis=1)
samples

In [ ]:
evaluator = EvaluatorEP(problem, building, out_dir='outputdir', err_dir='outputdir',epw='/home/ict4bd/larnaca.epw')
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.describe()

In [ ]:
_ = outputs.plot(x=objectives[0], y=objectives[1], style = 'o')
plt.xlabel(objectives[0])
plt.ylabel(objectives[1])

In [ ]:
outputs = outputs.sort_values(by=objectives[1])
ax=outputs.plot.bar(subplots=True,legend=None, figsize=(8,15))
outputs.to_csv(filename + '.csv', index = False)

In [ ]:
outputs